In [ ]:
import numpy as np
import commpy as cp
import scipy.signal as sig
import scipy.linalg as la
import math
import matplotlib.pyplot as plt

def modulate(data, mod_scheme='BPSK', demod=False):
    """  1. Modulates (or demodulates) data according to the modulation scheme """
    mod_schemes = ['BPSK', 'QPSK']
    data = data.flatten()
    if mod_scheme not in mod_schemes:
        raise ValueError('Unknown modulation scheme, please choose from: '+ ' '.join(mod_schemes))
    elif mod_scheme == 'QPSK':
        modulator = cp.modulation.QAMModem(4)
        if demod:
            return modulator.demodulate(data, "hard")
        return modulator.modulate(data)
    elif mod_scheme == 'BPSK':
        def bpsk_one(x):
            if demod:
                return 0 if x < 0 else 1
            return -1 if x==0 else 1
        bpsk = np.vectorize(bpsk_one)
        return bpsk(data)

def apply_channel(signal, channel_function):
    """  2. Convolves signal with channel_function """
    channel_output = sig.convolve(signal, channel_function, mode='full') # convolve input complex data with the channel transfer function
    return channel_output

def add_awgn_noise(signal, SNR_dB):
    """  3. Adds AWGN noise vector to signal  
            to generate a resulting signal vector y of specified SNR in dB
    """
    L=len(signal)
    SNR = 10**(SNR_dB/10.0) #SNR to linear scale
    Esym=np.sum(np.square(np.abs(signal)))/L #Calculate actual symbol energy
    N0=Esym/SNR; #Find the noise spectral density
    if(isinstance(signal[0], complex)):
        noiseSigma=np.sqrt(N0/2.0)#Standard deviation for AWGN Noise when x is complex
        n = noiseSigma*(np.random.randn(1,L)+1j*np.random.randn(1,L))#computed noise 
    else:
        noiseSigma = np.sqrt(N0);#Standard deviation for AWGN Noise when x is real
        n = noiseSigma*np.random.randn(1,L)#computed noise
    y = signal + n #received signal
    return y.flatten()

def num_bit_errs(in_bits, out_bits):
    total = 0
    for i in range(len(in_bits)):
        if in_bits[i] != out_bits[i]:
            total += 1
    return total


SIGNAL_LENGTH = 1000
PREAMBLE_LENGTH = 100
MOD_SCHEME = 'BPSK'
CHANNEL_LENGTH = 3
SNR = 5
def generate_data(num, preamble_bits=None):
    if preamble_bits is None:
        preamble_bits = np.random.randint(0,2, PREAMBLE_LENGTH)
    _p = []
    _x = []
    _y  = [] #
    _pb = preamble_bits
    _xb = [] #signal bits
    _c = [] #channel taps
    
    for i in range(num):
        # generate normalized channel function of consecutive taps
        channel_function = np.random.randn(CHANNEL_LENGTH) 
        channel_function = channel_function / np.linalg.norm(channel_function)
        _c.append(channel_function)
        # generate preamble 
        train_preamble_noisy = add_awgn_noise(
                            apply_channel(
                                modulate(preamble_bits, MOD_SCHEME), 
                                channel_function),
                            SNR) #len = PREAMBLE_LENGTH + CHANNEL_LENGTH-1
        _p.append(train_preamble_noisy)
        # generate message
        train_signal_bits = np.random.randint(0,2, SIGNAL_LENGTH)
        _xb.append(train_signal_bits)
        train_signal = modulate(train_signal_bits, MOD_SCHEME) #len = SIGNAL_LENGTH
        _y.append(train_signal)
        train_signal_noisy = add_awgn_noise(apply_channel(train_signal, channel_function), SNR)#len = SIGNAL_LENGTH + CHANNEL_LENGTH-1 
        _x.append(train_signal_noisy)
    return np.array(_p), np.array(_x), np.array(_y), np.array(_pb), np.array(_xb), np.array(_c)

In [ ]:
from keras.models import Sequential, Model
from keras import optimizers
from keras.layers import Input, Embedding, LSTM, Dense, Activation
import keras.layers as layers
import keras.backend as K
from keras.utils import to_categorical

NUM_ITERS = 100
BATCH_SIZE = 100
NUM_DATA = BATCH_SIZE * NUM_ITERS
NUM_TEST = 1000
  



# Inputs
preamble_input = Input(shape=(PREAMBLE_LENGTH+CHANNEL_LENGTH-1,), name='preamble')
preamble_input = LSTM(50, return_sequences=True)(preamble_input)
signal_input = Input(shape=(SIGNAL_LENGTH+CHANNEL_LENGTH-1,), name='signal')
signal_input = LSTM(500, return_sequences=True)(signal_input)
x = layers.concatenate([preamble_input, signal_input])
model.add(LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32))  # return a single vector of dimension 32
# Deep densely-connected network on top
x = Dense(500, activation='sigmoid')(x)
x = Dense(500, activation='sigmoid')(x)
x = Dense(500, activation='sigmoid')(x)
# Linear Layer
signal_output = Dense(SIGNAL_LENGTH, activation='softmax', name='signal_output')(x)
model = Model(inputs=[preamble_input, signal_input], outputs=[signal_output])

# sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
adam = optimizers.Adam(lr=0.05, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, clipnorm=1.)
def avg_bit_error(y_true, y_pred):
    zeroes = K.zeros(shape=(K.int_shape(y_true)[1]))
    return K.mean(K.cast(K.equal(K.greater(y_true, zeroes), K.greater(y_pred, zeroes)), dtype='float32'))

def loss_fcn(y_true, y_pred):
    return K.mean(K.square(y_true-y_pred))

model.compile(loss=loss_fcn, optimizer='adam', metrics=[avg_bit_error])

preamble_train, signal_train, signal_output_train, preamble_bits, _, _, = generate_data(NUM_DATA, preamble_bits)
model.fit({'preamble': preamble_train, 'signal': signal_train}, {'signal_output': signal_output_train.astype(float)}, epochs=100, batch_size=100)

# todo:
Add weight sharing: convolution
more data
    
    